# Turbofan Engine Degradation - NASA Prognostics Center of Excellence 

This notebooks presents the development of an algorithm to predict the Remaining Useful Life of Turbofan engines. By the end, the obtained ML model has an XX accuracy

It is divided into two main sections: Feature Engineering and Modelling.

Through FE section I'll prepare data for training, create new features and delete unnecessary ones, and transform and normalize data.
On modelling, I'll perform different tests with different models and datasets, choose the best version based on R2 score and optimize the hyperparameters with bayesian technique. 

---

## Imports and setup

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

import warnings

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

In [5]:
df_train_raw = pd.read_csv('./data/df_train_cleaned.csv', index_col=0)
df_test_raw = pd.read_csv('./data/df_test_cleaned.csv', index_col=0)

df_train_raw.head()

,unit_number,time,os_1,os_2,os_3,sm_1,sm_2,sm_3,sm_4,sm_5,sm_6,sm_7,sm_8,sm_9,sm_10,sm_11,sm_12,sm_13,sm_14,sm_15,sm_16,sm_17,sm_18,sm_19,sm_20,sm_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


---
## Feature Engineering
Along this section, data will be transformed for obtaining better accuracy on the model.
The problems we'll try to solve and the approach are the following ones:

1. Not equally distributed data --> Perform normalization;
    
2. Columns with constant values --> Remove this column as they don't add information;
    
3. Past measurements helps on predicting RULs? --> Add lagged data

In [6]:
def remove_cols(df, cols):
    """
    Drop selected cols.

    Parameters:
    - df (pd.DataFrame): the input DataFrame containing the time series data.
    - cols (list): list of cols to drop.

    Returns
        A new DataFrame with the cols dropped.
    """
    df.drop(columns=cols, inplace=True)
    return df

In [7]:
def normalize_data(df, scaler):
    """
    Normalize dataframe.

    Parameters:
    - df (pd.DataFrame): the input DataFrame containing the time series data.
    - scaler: model to normalize the data.

    Returns
        A new DataFrame with the normalized features.
    """
    normalized_data = scaler.fit_transform(df.drop(columns='RUL'))
    df_norm = pd.DataFrame(normalized_data, columns=df.drop(columns='RUL').columns)
    df_norm['RUL'] = df['RUL']
    return df_norm

In [8]:
def add_lagged_measures(df, qtd):
    """
    Create lagged features with the mean value of 'qtd' past rows.

    Parameters:
    - df (pd.DataFrame): the input DataFrame containing the time series data.
    - qtd (int): the number of past rows to include in the lagged features.

    Returns
        A new DataFrame with the lagged features added.
    """
    df_lagged = df.copy()
    cols_sm = filter_col = [col for col in df_lagged if col.startswith('sm_')]

    for sensor in cols_sm:
      for i in range(1, qtd+1):
          df_lagged[f'L_{sensor}_{i}'] = df_lagged[f'{sensor}'].shift(i).rolling(window=i).mean()

    # Drop the rows with missing values
    df_lagged.dropna(inplace=True)
    return df_lagged

# 2. Modelling
1. Define metrics and evaluation function
2. Models
3. Cross validation


In [9]:
def prepare_train_test_dfs(df_train, df_test):
    """
    Split and organize the data for trainning.

    Parameters:
    - df_train (pd.DataFrame)
    - df_test (pd.DataFrame)

    Returns
        The data splitted.
    """
    X_train = df_train.drop(columns='RUL')
    y_train = df_train['RUL']

    X_test = df_test.drop(columns='RUL')
    y_test = df_test['RUL']
    return X_train, y_train, X_test, y_test

In [10]:
def test_models(df_train, df_test, models):
    """
    Iterates over a list of models and select the one with highest accuracy..

    Parameters:
    - df_train (pd.DataFrame)
    - df_test (pd.DataFrame)
    - model (tuple): selected models

    Returns
        Results of perfomance of each model and the best one.
    """
    X_train, y_train, X_test, y_test = prepare_train_test_dfs(df_train, df_test)
    
    model_results = []
    best_rmse = 100
    for name, model in models:
        regressor = model.fit(X_train, y_train)
        predicted = regressor.predict(X_test)
        cv_score = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
        cv2_rmse = np.sqrt(cv_score)
    
        if cv2_rmse.mean() < best_rmse:
            best_model = model
        
        model_results.append([name, cv_score.mean(), cv2_rmse.mean()])

    df_model_results = pd.DataFrame(model_results, columns=['model', 'cv_score_r2', 'cv_score_rmse'])
    
    return df_model_results, best_model   

In [11]:
models = [
      ('LinReg', LinearRegression()), 
      ('RF', RandomForestRegressor()),
      ('SVR', SVR()), 
      ('XGB', XGBRegressor())
    ]

In [12]:
df_results, model = test_models(df_train_raw, df_test_raw, models)
df_results, model

[169.2392208  174.89789473 161.41324018 ...  51.1843808   48.20099162
  32.41571624]
[192.15 200.44 187.29 ...  24.13  22.39  16.81]
[104.20218973 104.15845018 104.14510509 ... 101.11804733 101.07283447
 101.03827535]
[189.33284   208.07828   188.0731    ...  27.2548     17.822811
  10.3357935]


(    model  cv_score_r2  cv_score_rmse
 0  LinReg     0.645533       0.802104
 1      RF     0.568721       0.749565
 2     SVR    -0.009119            NaN
 3     XGB     0.609482       0.777592,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...))

---
## Testing combinations of data transformation
1. Without unnecessary columns
2. Without unnecessary columns and normalize others
3. Without unnecessary columns, normalize others and added lagged values

In [13]:
def select_df(df_train, df_test, model):
    """
    Pipeline of trainning and evaluateing the model and dataset.

    Parameters:
    - df_train (pd.DataFrame)
    - df_test (pd.DataFrame)
    - model : selected model

    Returns
        Results of perfomance of the model.
    """
    X_train, y_train, X_test, y_test = prepare_train_test_dfs(df_train, df_test)
    regressor = model.fit(X_train, y_train)
    predicted = regressor.predict(X_test)

    cv_score = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
    mse = mean_squared_error(y_test, predicted)
    rmse = np.sqrt(mse)
    cv2_rmse = np.sqrt(cv_score).mean()
    
    results = [cv2_rmse, cv_score]
    return results 

### 1. Remove unnecessary columns

In [14]:
df_train = df_train_raw.copy()
df_test = df_test_raw.copy()

In [15]:
unnecessary_cols = ['unit_number', 'os_3', 'sm_1', 'sm_5', 'sm_10', 'sm_16', 'sm_18', 'sm_19']

In [16]:
train_no_cols = df_train.copy().pipe(remove_cols, unnecessary_cols)
test_no_cols = df_test.copy().pipe(remove_cols, unnecessary_cols)

In [17]:
result_norm_no_cols = select_df(train_no_cols, test_no_cols, model)
print(result_norm_no_cols[0])

0.8119281486953694


### 2. Remove unnecessary columns and perform normalization

In [18]:
train_norm_no_cols = (df_train.copy().pipe(remove_cols, unnecessary_cols)
                               .pipe(normalize_data, StandardScaler()))
test_norm_no_cols = (df_test.copy().pipe(remove_cols, unnecessary_cols)
                             .pipe(normalize_data, StandardScaler()))

In [19]:
result_norm_no_cols = select_df(train_norm_no_cols, test_norm_no_cols, model)
print(result_norm_no_cols[0])

0.8117736895146741


### 3. Remove unnecessary columns, perform normalization and add lagged data

In [20]:
NUM_LAGGED_ROWS = 1

In [21]:
train_lag_norm_no_cols = (df_train.copy().pipe(remove_cols, unnecessary_cols)
                               .pipe(normalize_data, StandardScaler())
                               .pipe(add_lagged_measures, NUM_LAGGED_ROWS))
test_lag_norm_no_cols = (df_test.copy().pipe(remove_cols, unnecessary_cols)
                             .pipe(normalize_data, StandardScaler())
                             .pipe(add_lagged_measures, NUM_LAGGED_ROWS))

In [22]:
result_norm_no_cols = select_df(train_lag_norm_no_cols, test_lag_norm_no_cols, model)
print(result_norm_no_cols[0])

0.8114351240939328


#### Find the best amount of lagged measurements to model perfomance

In [23]:
df_lag_optmization = pd.DataFrame(columns=['days', 'rmse', 'delta'])
df_train_lag_opt = train_norm_no_cols.copy()
df_test_lag_opt = test_norm_no_cols.copy()

past_rmse = result_norm_no_cols[0]
for i in range(1,25):
    NUM_LAGGED_ROWS = i
    df_train_lagged = df_train_lag_opt.pipe(add_lagged_measures, NUM_LAGGED_ROWS)
    df_test_lagged = df_test_lag_opt.pipe(add_lagged_measures, NUM_LAGGED_ROWS)
    
    rmse_atual = select_df(df_train_lagged, df_test_lagged, model)
    delta = rmse_atual[0] - past_rmse
    print(NUM_LAGGED_ROWS)
    print(delta)
    df_actual_lag = pd.DataFrame({'days': NUM_LAGGED_ROWS, 'rmse': rmse_atual[0], 'delta': delta}, index=[0])
    past_rmse = rmse_atual[0]
    df_lag_optmization = pd.concat([df_lag_optmization, df_actual_lag])

df_lag_optmization

1
0.0
2
-0.0032371469398746466
3
-0.007443144578898386
4
-0.000628285427333175
5
0.0021745834404879316
6
-0.004326854538539204
7
-0.001521085224907348
8
-0.0010531438000520588
9
-0.001321087414721811
10
-0.0011944088387355611
11
0.006733656614853922
12
-0.0012284470073228304
13
0.003684382793294194
14
0.003729776385174799
15
0.003116734299477786
16
-0.0039493481751234905
17
0.0029570198805801695
18
0.003118144703433523


KeyboardInterrupt: 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,4))
sns.lineplot(y=f'delta', x='days', palette='hls', data=df_lag_optmization, ax=ax)

## Hyperparameters optimization 
After decideing which model and dataset, we'll improve model accuracy by utilizing Bayesian Optmization. 
This technique Bayesian optimization works by constructing a posterior distribution of functions (gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not.
As you iterate over and over, the algorithm balances its needs of exploration and exploitation taking into account what it knows about the target function. At each step a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with a exploration strategy (such as UCB (Upper Confidence Bound), or EI (Expected Improvement)), are used to determine the next point that should be explored.





In [ ]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = LinearRegression() ###### 
    model.train(x=X_train, y=_train, training_frame=train)
    return -model.rmse()

bounds = {
    'max_depth':(5,10),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=50)
optimizer.max

In [ ]:
# https://medium.com/@beniciowg/como-tunar-hiperpar%C3%A2metros-com-otimiza%C3%A7%C3%A3o-bayesiana-5687fd51370f

# importando o BayesSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
# instanciando o algoritmo de classificação XGBoost
clf = RandomForestRegressor()
# definindo as faixas de valores a serem testadas para cada hiperparâmetro a ser otimizado
space = {
    'max_depth':(5,10)
}
# instanciando o algoritmo de otimização e definindo alguns parâmetros dele
opt = BayesSearchCV(clf, space, n_iter=5, random_state=42, cv=5,
                    return_train_score=True, scoring='r2', refit=True)
# executando o modelo de otimização
opt.fit(X_train, y_train)
# aplicando o modelo nos dados de teste
y_pred_opt = opt.predict(X_test)
mse = mean_squared_error(y_test, y_pred_opt)
# imprimindo os resultados de avaliação
print(mse)